In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2333707,2022-04-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2333708,2022-04-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2333709,2022-04-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2333710,2022-04-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
59923,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59925,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59927,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59929,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59931,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1412741,2022-04-21,Arkansas,Arkansas,5001,5683,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1412743,2022-04-22,Arkansas,Arkansas,5001,5683,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1412745,2022-04-23,Arkansas,Arkansas,5001,5683,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1412747,2022-04-24,Arkansas,Arkansas,5001,5683,79.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2333707,2022-04-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2333708,2022-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2333709,2022-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2333710,2022-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-04-25') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
825,2022-04-25,Snohomish,Washington,153901,1136.00,53061,WA,County,822083,18720.86,138.19
1648,2022-04-25,Cook,Illinois,1154229,14887.00,17031,IL,County,5150233,22411.20,289.05
2470,2022-04-25,Orange,California,591778,6990.00,6059,CA,County,3175692,18634.62,220.11
3291,2022-04-25,Maricopa,Arizona,1275525,17103.00,4013,AZ,County,4485414,28437.17,381.30
4112,2022-04-25,Los Angeles,California,2864988,31938.00,6037,CA,County,10039107,28538.28,318.14
...,...,...,...,...,...,...,...,...,...,...,...
2331595,2022-04-25,Wheeler,Oregon,244,3.00,41069,OR,County,1332,18318.32,225.23
2332155,2022-04-25,King,Texas,51,0.00,48269,TX,County,272,18750.00,0.00
2332684,2022-04-25,Esmeralda,Nevada,109,3.00,32009,NV,County,873,12485.68,343.64
2333209,2022-04-25,Loving,Texas,186,1.00,48301,TX,County,169,110059.17,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
825,2022-04-25,Snohomish,Washington,153901,1136.00,53061,WA,County,822083,18720.86,138.19,138.19,18720.86
1648,2022-04-25,Cook,Illinois,1154229,14887.00,17031,IL,County,5150233,22411.20,289.05,289.05,22411.20
2470,2022-04-25,Orange,California,591778,6990.00,6059,CA,County,3175692,18634.62,220.11,220.11,18634.62
3291,2022-04-25,Maricopa,Arizona,1275525,17103.00,4013,AZ,County,4485414,28437.17,381.30,381.30,28437.17
4112,2022-04-25,Los Angeles,California,2864988,31938.00,6037,CA,County,10039107,28538.28,318.14,318.14,28538.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331595,2022-04-25,Wheeler,Oregon,244,3.00,41069,OR,County,1332,18318.32,225.23,225.23,18318.32
2332155,2022-04-25,King,Texas,51,0.00,48269,TX,County,272,18750.00,0.00,0.00,18750.00
2332684,2022-04-25,Esmeralda,Nevada,109,3.00,32009,NV,County,873,12485.68,343.64,343.64,12485.68
2333209,2022-04-25,Loving,Texas,186,1.00,48301,TX,County,169,110059.17,591.72,591.72,110059.17


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1477934,2022-04-25,Hale,Alabama,4723,106.00,1065,AL,County,14651,32236.71,723.50,723.50,32236.71,1
1246418,2022-04-25,Winston,Alabama,7565,131.00,1133,AL,County,23629,32015.74,554.40,554.40,32015.74,2
856116,2022-04-25,Franklin,Alabama,9878,138.00,1059,AL,County,31362,31496.72,440.02,440.02,31496.72,3
1018713,2022-04-25,Clay,Alabama,4084,82.00,1027,AL,County,13235,30857.57,619.57,619.57,30857.57,4
1477177,2022-04-25,Fayette,Alabama,4984,96.00,1057,AL,County,16302,30572.94,588.88,588.88,30572.94,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996164,2022-04-25,Big Horn,Wyoming,2580,60.00,56003,WY,County,11790,21882.95,508.91,508.91,21882.95,19
1242618,2022-04-25,Johnson,Wyoming,1845,19.00,56019,WY,County,8445,21847.25,224.99,224.99,21847.25,20
1757361,2022-04-25,Lincoln,Wyoming,3972,31.00,56023,WY,County,19830,20030.26,156.33,156.33,20030.26,21
1410475,2022-04-25,Sublette,Wyoming,1922,27.00,56035,WY,County,9831,19550.40,274.64,274.64,19550.40,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1153581,2022-04-25,Lowndes,Alabama,2615,76.00,1085,AL,County,9726,26886.70,781.41,781.41,26886.70,31,1
1477934,2022-04-25,Hale,Alabama,4723,106.00,1065,AL,County,14651,32236.71,723.50,723.50,32236.71,1,2
1152059,2022-04-25,Crenshaw,Alabama,3816,99.00,1041,AL,County,13772,27708.39,718.85,718.85,27708.39,25,3
487390,2022-04-25,Walker,Alabama,18732,443.00,1127,AL,County,63521,29489.46,697.41,697.41,29489.46,9,4
1016427,2022-04-25,Butler,Alabama,5063,129.00,1013,AL,County,19448,26033.53,663.31,663.31,26033.53,37,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242618,2022-04-25,Johnson,Wyoming,1845,19.00,56019,WY,County,8445,21847.25,224.99,224.99,21847.25,20,19
1712934,2022-04-25,Uinta,Wyoming,5625,39.00,56041,WY,County,20226,27810.74,192.82,192.82,27810.74,7,20
1757361,2022-04-25,Lincoln,Wyoming,3972,31.00,56023,WY,County,19830,20030.26,156.33,156.33,20030.26,21,21
1149015,2022-04-25,Albany,Wyoming,9894,48.00,56001,WY,County,38880,25447.53,123.46,123.46,25447.53,14,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,138.19,18720.86,31,17
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,138.19,18720.86,31,17
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,138.19,18720.86,31,17
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,138.19,18720.86,31,17
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,138.19,18720.86,31,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328331,2022-04-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2328332,2022-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2328333,2022-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2328334,2022-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1471802,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32236.71,2,1,1.00,nan
1471803,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32236.71,2,1,0.00,nan
1471804,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32236.71,2,1,0.00,nan
1471805,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32236.71,2,1,0.00,nan
1471806,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,723.50,32236.71,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787325,2022-04-21,Crook,Wyoming,1347,28.00,56011,WY,County,7584,17761.08,369.20,369.20,17761.08,9,23,0.00,0.07
1787326,2022-04-22,Crook,Wyoming,1347,28.00,56011,WY,County,7584,17761.08,369.20,369.20,17761.08,9,23,0.00,0.14
1787327,2022-04-23,Crook,Wyoming,1347,28.00,56011,WY,County,7584,17761.08,369.20,369.20,17761.08,9,23,0.00,0.14
1787328,2022-04-24,Crook,Wyoming,1347,28.00,56011,WY,County,7584,17761.08,369.20,369.20,17761.08,9,23,0.00,0.14


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1148963,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26886.70,1,31,1.00,0.21,0.00,nan
1148964,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26886.70,1,31,0.00,0.21,0.00,nan
1148965,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26886.70,1,31,0.00,0.21,0.00,nan
1148966,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26886.70,1,31,0.00,0.21,0.00,nan
1148967,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26886.70,1,31,0.00,0.21,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481986,2022-04-21,Teton,Wyoming,9975,16.00,56039,WY,County,23464,42511.93,68.19,68.19,42511.93,23,1,0.00,3.64,0.00,0.00
481987,2022-04-22,Teton,Wyoming,9975,16.00,56039,WY,County,23464,42511.93,68.19,68.19,42511.93,23,1,0.00,3.07,0.00,0.00
481988,2022-04-23,Teton,Wyoming,9975,16.00,56039,WY,County,23464,42511.93,68.19,68.19,42511.93,23,1,0.00,3.07,0.00,0.00
481989,2022-04-24,Teton,Wyoming,9975,16.00,56039,WY,County,23464,42511.93,68.19,68.19,42511.93,23,1,0.00,3.07,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-04-25') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
582590,2022-04-25,Imperial,California,66879,909.00,6025,CA,County,181215,36905.89,501.61,501.61,36905.89,1,1,0.00,34.57,0.00,0.00
88760,2022-04-25,Shasta,California,36776,626.00,6089,CA,County,180080,20422.03,347.62,347.62,20422.03,2,29,0.00,0.64,0.00,0.86
1411163,2022-04-25,Tuolumne,California,13520,181.00,6109,CA,County,54478,24817.36,332.24,332.24,24817.36,3,13,0.00,3.00,0.00,0.00
288672,2022-04-25,San Bernardino,California,591785,7047.00,6071,CA,County,2180085,27145.04,323.24,323.24,27145.04,4,7,0.00,324.21,0.00,11.07
4112,2022-04-25,Los Angeles,California,2864988,31938.00,6037,CA,County,10039107,28538.28,318.14,318.14,28538.28,5,5,4485.00,1284.36,14.00,9.36
159438,2022-04-25,Tulare,California,135110,1453.00,6107,CA,County,466195,28981.43,311.67,311.67,28981.43,6,4,0.00,99.93,7.00,2.50
1024761,2022-04-25,Inyo,California,4584,56.00,6027,CA,County,18039,25411.61,310.44,310.44,25411.61,7,11,0.00,2.07,0.00,0.00
1712076,2022-04-25,Tehama,California,14696,201.00,6103,CA,County,65084,22580.05,308.83,308.83,22580.05,8,20,0.00,2.21,0.00,0.00
128401,2022-04-25,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,9,14,0.00,0.00,0.00,0.00
927877,2022-04-25,Merced,California,72417,820.00,6047,CA,County,277680,26079.30,295.30,295.30,26079.30,10,9,58.00,16.29,0.00,0.36


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
582590,2022-04-25,Imperial,California,66879,909.00,6025,CA,County,181215,36905.89,501.61,501.61,36905.89,1,1,0.00,34.57,0.00,0.00
1244840,2022-04-25,Kings,California,55220,444.00,6031,CA,County,152940,36105.66,290.31,290.31,36105.66,12,2,0.00,5.50,0.00,0.07
2194675,2022-04-25,Lassen,California,10725,69.00,6035,CA,County,30573,35079.97,225.69,225.69,35079.97,20,3,0.00,0.29,0.00,0.00
159438,2022-04-25,Tulare,California,135110,1453.00,6107,CA,County,466195,28981.43,311.67,311.67,28981.43,6,4,0.00,99.93,7.00,2.50
4112,2022-04-25,Los Angeles,California,2864988,31938.00,6037,CA,County,10039107,28538.28,318.14,318.14,28538.28,5,5,4485.00,1284.36,14.00,9.36
58362,2022-04-25,Madera,California,43430,354.00,6039,CA,County,157327,27604.92,225.01,225.01,27604.92,21,6,-15.00,0.50,0.00,0.00
288672,2022-04-25,San Bernardino,California,591785,7047.00,6071,CA,County,2180085,27145.04,323.24,323.24,27145.04,4,7,0.00,324.21,0.00,11.07
363501,2022-04-25,Kern,California,243186,2356.00,6029,CA,County,900202,27014.60,261.72,261.72,27014.60,15,8,0.00,36.00,0.00,2.57
927877,2022-04-25,Merced,California,72417,820.00,6047,CA,County,277680,26079.30,295.30,295.30,26079.30,10,9,58.00,16.29,0.00,0.36
57582,2022-04-25,Fresno,California,255391,2744.00,6019,CA,County,999101,25562.08,274.65,274.65,25562.08,13,10,0.00,111.93,0.00,0.93


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
582590,2022-04-25,Imperial,California,66879,909.00,6025,CA,County,181215,36905.89,501.61,501.61,36905.89,1,1,0.00,34.57,0.00,0.00
88760,2022-04-25,Shasta,California,36776,626.00,6089,CA,County,180080,20422.03,347.62,347.62,20422.03,2,29,0.00,0.64,0.00,0.86
1411163,2022-04-25,Tuolumne,California,13520,181.00,6109,CA,County,54478,24817.36,332.24,332.24,24817.36,3,13,0.00,3.00,0.00,0.00
288672,2022-04-25,San Bernardino,California,591785,7047.00,6071,CA,County,2180085,27145.04,323.24,323.24,27145.04,4,7,0.00,324.21,0.00,11.07
4112,2022-04-25,Los Angeles,California,2864988,31938.00,6037,CA,County,10039107,28538.28,318.14,318.14,28538.28,5,5,4485.00,1284.36,14.00,9.36
159438,2022-04-25,Tulare,California,135110,1453.00,6107,CA,County,466195,28981.43,311.67,311.67,28981.43,6,4,0.00,99.93,7.00,2.50
1024761,2022-04-25,Inyo,California,4584,56.00,6027,CA,County,18039,25411.61,310.44,310.44,25411.61,7,11,0.00,2.07,0.00,0.00
1712076,2022-04-25,Tehama,California,14696,201.00,6103,CA,County,65084,22580.05,308.83,308.83,22580.05,8,20,0.00,2.21,0.00,0.00
128401,2022-04-25,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,9,14,0.00,0.00,0.00,0.00
927877,2022-04-25,Merced,California,72417,820.00,6047,CA,County,277680,26079.30,295.30,295.30,26079.30,10,9,58.00,16.29,0.00,0.36


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,4.00,0.43
1,03/21/20,Imperial,4,2.21,0.00,0.43
2,03/22/20,Imperial,4,2.21,0.00,0.36
3,03/23/20,Imperial,4,2.21,0.00,0.36
4,03/24/20,Imperial,9,4.97,5.00,0.71
...,...,...,...,...,...,...
7686,04/21/22,Madera,43445,27614.46,0.00,2.07
7687,04/22/22,Madera,43445,27614.46,0.00,1.57
7688,04/23/22,Madera,43445,27614.46,0.00,1.57
7689,04/24/22,Madera,43445,27614.46,0.00,1.57


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...
7720,04/21/22,Merced,820.00,295.30,0.00,0.36
7721,04/22/22,Merced,820.00,295.30,0.00,0.36
7722,04/23/22,Merced,820.00,295.30,0.00,0.36
7723,04/24/22,Merced,820.00,295.30,0.00,0.36


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)